In [1]:
import pandas as pd
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors
from fuzzywuzzy import process

/Users/philipdecanesie/.local/share/virtualenvs/Machine-learning-Philip-de-Canesie-tsBDOfXA/lib/python3.9/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [2]:
movies_path = "../Data/ml-latest-small/movies.csv"
ratings_path = "../Data/ml-latest-small/ratings.csv"

df_movies = pd.read_csv(movies_path, usecols=["movieId", "title"], dtype= {"movieId": "int32", "title": "str"})
df_ratings = pd.read_csv(ratings_path, usecols=["userId", "movieId", "rating"], dtype= {"userId": "int32", "movieId": "int32", "rating": "float32"})

In [3]:
df_ratings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100836 entries, 0 to 100835
Data columns (total 3 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   userId   100836 non-null  int32  
 1   movieId  100836 non-null  int32  
 2   rating   100836 non-null  float32
dtypes: float32(1), int32(2)
memory usage: 1.2 MB


In [4]:
# spare matrix 

movies_users = df_ratings.pivot(index="movieId", columns="userId", values="rating").fillna(0)
mat_movies_users = csr_matrix(movies_users.values)

In [5]:
model_KNN = NearestNeighbors(metric= "cosine", algorithm= "brute", n_neighbors= 20)

In [6]:
model_KNN.fit(mat_movies_users)

NearestNeighbors(algorithm='brute', metric='cosine', n_neighbors=20)

In [7]:
def recommender(movie_name, data, model, n_recommendations):

    model.fit(data)
    index = process.extractOne(movie_name, df_movies["title"])[2]
    print(f"Movie Selected: {df_movies['title'][index]}, Index: {index}")
    print("Serching for recommendations....")

    distances, indices = model.kneighbors(data[index], n_neighbors = n_recommendations)
    
    for i in indices:

        print(df_movies["title"][i].where(i!=index))

    
recommender("toy story", mat_movies_users, model_KNN, 20)

Movie Selected: Toy Story (1995), Index: 0
Serching for recommendations....
0                                                     NaN
2353                                 'night Mother (1986)
418                                  Jurassic Park (1993)
615                  Independence Day (a.k.a. ID4) (1996)
224             Star Wars: Episode IV - A New Hope (1977)
314                                   Forrest Gump (1994)
322                                 Lion King, The (1994)
910     Once Upon a Time in the West (C'era una volta ...
546                            Mission: Impossible (1996)
963                                           Diva (1981)
968                           Arsenic and Old Lace (1944)
3189            Rififi (Du rififi chez les hommes) (1955)
506                                        Aladdin (1992)
123                                      Apollo 13 (1995)
257                                   Pulp Fiction (1994)
897                 Cheech and Chong's Up in Smoke (19